#  1）建立对象与导入原始数据

In [1]:
######建立对象
import sys
sys.path.append('E:\\python-dq\\py\\work')
import imp

import dq_work.goods.goods_1 as goods


imp.reload(goods)#再次引入模块，默认只有一次，调试程序时有用

#建立对象
###建立对象的相关参数变量

company='上海'
date_Range='202301'
date_Range_all='20230101-0127'
in_file_path='e:/python-dq/work_data/goods_data/in/'
out_file_path='e:/python-dq/work_data/goods_data/out/'
public_path='e:/python-dq/work_data/public_data/out/'

all_sku_goods_obj=goods.all_sku_goods(company,date_Range,date_Range_all,in_file_path,out_file_path,public_path)


In [2]:
##导入销售原始数据
in_file1='03_业务报表-销售报表-销售-'+company+ '-'+date_Range_all+'.csv'
df_原始销售_all= \
all_sku_goods_obj.load_data_all_sku_GoodPath(in_file1)

df_原始销售_all[['供应商业务码']]=df_原始销售_all[['供应商业务码']].astype(str) 
df_原始销售_all['供应商业务码']=df_原始销售_all['供应商业务码'].str.strip()


导入all sku GoodPath 原始数据 over


In [3]:
##导入券原始数据
in_file2='01-用劵分摊-商品层级-原版-all-data-'+company+ '-'+date_Range_all+'.csv'
df_原始券_all= \
all_sku_goods_obj.load_data_all_sku_PublicPath(in_file2)

df_原始券_all[['供应商业务码']]=df_原始券_all[['供应商业务码']].astype(str) 
df_原始券_all['供应商业务码']=df_原始券_all['供应商业务码'].str.strip()


导入all sku PublicPath 原始数据 over


In [4]:
df_原始券_all.loc[:,['万家承担金额（去税）']].sum()

万家承担金额（去税）    1.076276e+07
dtype: float64

In [5]:
##导入类别数据
in_file5='B-分析-品类基础资料.csv'
df_类别= \
all_sku_goods_obj.load_data_all_sku_PublicPath(in_file5)

导入all sku PublicPath 原始数据 over


In [6]:
print(df_原始销售_all.columns)
print(df_原始券_all.columns)

Index(['开始时间', '结束时间', '省级公司名称', '模块编码', '模块名称', '品类编码', '品类名称', '大类编码',
       '大类名称', '中类编码', '中类名称', '小类编码', '小类名称', '商品编码', '商品条码', '商品名称', '销售单位',
       '品牌名称', '供应商业务码', '供应商名称', '渠道', '子渠道合计', '平台合计', '销售金额', '销售数量',
       '销售净额(折前)', '销售净额', '销售成本', '折扣净额', '补差', '生鲜损耗', '毛利额'],
      dtype='object')
Index(['日期', '省级公司名称', '业态名称', '采购管理区域名称', '营运区域', '新旧店标识', '门店编码', '门店DC名称',
       '供应商业务码', '供应商名称', '部门名称', '大类编码', '大类名称', '小类编码', '小类名称', '商品编码',
       '商品条码', '商品名称', '品牌名称', '渠道', '平台', '销项税率', '券类型编码', '券类型名称', '券号',
       '优惠类型', '小优惠类型', '用券总金额（去税）', '平台承担金额（去税）', '供应商承担金额（去税）',
       '万家承担金额（去税）'],
      dtype='object')


In [7]:
##################梳理销售df的字段
sale_columns_list=[  '大类编码',
       '大类名称', '中类编码', '中类名称', '小类编码', '小类名称', '商品编码','供应商业务码', '渠道', '销售金额', '销售数量',
       '销售净额', '销售成本', '折扣净额', '补差', '生鲜损耗', '毛利额']
#选择字段
df_销售=df_原始销售_all.loc[:,sale_columns_list]
#批量修改dataframe的字段名称
sale_columns_name=[ '大类编码',
       '大类名称', '中类编码', '中类名称', '小类编码', '小类名称', '商品编码','供应商业务码', '渠道', '销售金额', '销售数量',
       '销售净额', '销售成本', '折扣净额', '补差', '生鲜损耗', '销售毛利额']
df_销售.columns=sale_columns_name

###################梳理券df的字段
coupon_columns_list=[ '渠道', '大类编码','商品编码', '供应商业务码', '用券总金额（去税）', '供应商承担金额（去税）', '平台承担金额（去税）', '万家承担金额（去税）']
#选择字段
df_券=df_原始券_all.loc[:,coupon_columns_list]

#批量修改dataframe的字段名称
#coupon_columns_name=[ '渠道', '大类编码','总用券金额', '供应商承担金额', '平台承担金额', '万家承担金额']
#df_券.columns=coupon_columns_name

print('df字段梳理完成！')


df字段梳理完成！


# 2）计算数据

In [8]:
########计算########
#合并sku的销售数据与券数据
df_sale_coupons=all_sku_goods_obj.sale_coupons_join_all_sku(df_销售,df_券)

all sku的销售数据与券数据合并完成
销售净额    2.202665e+08
dtype: float64
销售净额    2.202665e+08
dtype: float64
万家承担金额（去税）    1.076276e+07
dtype: float64


In [9]:
#创建商品sku主档数据
df_商品主档=all_sku_goods_obj.create_sku_master_data(df_原始销售_all,df_原始券_all)

In [10]:
#创建供应商主档数据
df_供应商主档=all_sku_goods_obj.create_supplier_master_data(df_原始销售_all,df_原始券_all)

# 精加工，导出到mysql

In [11]:
#精加工大类与单品输出的DF
df_out2=all_sku_goods_obj.finish_machining_outfile_all_sku(df_sale_coupons,df_商品主档,df_类别)

#精加工供应商输出的DF
df_out3=all_sku_goods_obj.finish_machining_outfile_supplier(df_out2,df_供应商主档)

精加工 dataframe over
精加工供应商 over


In [12]:
#将结果落库mysql
#落库的table
in_table1='all_sku_sale_coupons' + date_Range
all_sku_goods_obj.input_mysql_all_sku(df_out3,in_table1)


all sku input mysql
